# Neutron Data Quick Start

In this tutorial demonstrates how neutron-scattering data can be loaded, visualized, and manipulated with generic functionality from `scipp` as well as neutron-specific functionality from `scipp.neutron`.

In [1]:
import numpy as np
import scipp as sc
from scipp.plot import plot

### Loading Nexus files

Loading Nexus files requires Mantid.
See, e.g., [Installation](https://scipp.github.io/getting-started/installation.html) on how to install scipp and mantid with conda.
We are using two files in this tutorial,
[PG3_4844_event.nxs](http://198.74.56.37/ftp/external-data/MD5/d5ae38871d0a09a28ae01f85d969de1e)
and
[PG3_4866_event.nxs](http://198.74.56.37/ftp/external-data/MD5/3d543bc6a646e622b3f4542bc3435e7e).
Both are available as part of Mantid's test data.

We start by loading two files and insert them into a dataset.

In [2]:
events = sc.Dataset()
events['sample'] = sc.neutron.load(filename='PG3_4844_event.nxs',
                                   load_pulse_times=False,
                                   mantid_args={'LoadMonitors': True})
events['vanadium'] = sc.neutron.load(filename='PG3_4866_event.nxs',
                                     load_pulse_times=False)

ValueError: Problem when setting Filename. This is the detailed error description: Invalid value for property Filename (list of str lists) from string "PG3_4844_event.nxs": When setting value of property "Slave": File "PG3_4844_event.nxs" not found
If the file has been found but you got this error, you might not have read permissions or the file might be corrupted.
If the file has not been found, you might have forgotten to add its location in the data search directories.

The optional `mantid_args` dict is forwarded to the Mantid algorithm used for loading the files &ndash; in this case [LoadEventNexus](https://docs.mantidproject.org/nightly/algorithms/LoadEventNexus-v1.html) &ndash; and can be used to control, e.g., which part of a file to load.
Here we request loading monitors, which Mantid does not load by default.
The resulting dataset looks as follows:

In [ ]:
sc.show(events)

### Instrument view

Scipp provides a rudimentary version of the Mantid [instrument view](https://www.mantidproject.org/MantidPlot:_Instrument_View), which can be used to take a quick look at the neutron counts on the detector panels in 3D space or using various cylindrical and spherical projections

In [ ]:
sc.neutron.instrument_view(events)

### Plot against scattering angle $\theta$ using `groupby`

*This is not an essential step and can be skipped.*

Plotting raw data directly yields a hard-to-interpret figure.
We can obtain something more useful by "binning" the spectrum axis based on its $\theta$ value, using the split-apply-combine approach provided by `groupby`:

In [ ]:
events.coords['scattering_angle'] = sc.neutron.scattering_angle(events)
theta_bins = sc.Variable(['scattering_angle'],
                         unit=sc.units.rad,
                         values=np.linspace(0.0, np.pi/2, num=2000))

In [ ]:
# Note: Use `sum` instead of `flatten` when working with dense (histogrammed) data
theta_events = sc.groupby(
    events,
    'scattering_angle',
    bins=theta_bins).flatten('spectrum')

In [ ]:
theta_events

In [ ]:
plot(theta_events, bins=np.linspace(0.0, 17000.0, 1000))

### Unit conversion

*Note: We are back to working with `events`, not `theta_events`.*

`scipp.neutron` provides means to convert between units (dimensions) related to time-of-flight.
The loaded raw data has `Dim.Tof`, and we convert to interplanar lattice spacing (d-spacing):

In [ ]:
dspacing_events = sc.neutron.convert(events, 'tof', 'd-spacing')
dspacing_events

### Neutron monitors

*Processing after this section does not continue based on the monitor-normalized data produced here.
This section could thus be skipped.*

If available, neutron monitors are stored as attributes of a data array:

In [ ]:
mon = events['sample'].attrs['monitor1'].value
mon

The monitor could, e.g., be used to normalize the data.
To do so, both data and monitor need to be converted to a unit other than time-of-flight, e.g., wavelength or energy.
We also rebin the monitor since the original binning is very fine:

In [ ]:
sample = sc.neutron.convert(events['sample'], 'tof', 'wavelength')
mon = sc.neutron.convert(mon, 'tof', 'wavelength')
mon = sc.rebin(
    mon,
    'wavelength',
    sc.Variable(['wavelength'], unit=sc.units.angstrom, values=np.linspace(0, 1, num=1000)))
mon

In this case the sample data is sparse (event-mode data), whereas the monitor is a histogram.
Multiplication and division operations for such cases are supported by modifying the weights (values) for each event:

In [ ]:
sample_over_mon = sample / mon
sample_over_mon

Finally, we can plot the sparse data with on-the-fly binning:

In [ ]:
plot(sample_over_mon, bins=np.linspace(0, 1, num=1000))

### From events to histogram

*Note: We are continuing here with data that has not been normalized to the monitors.*

We histogram the event data:

In [ ]:
dspacing_bins = sc.Variable(
    ['d-spacing'],
    values=np.arange(0.3, 2.0, 0.001),
    unit=sc.units.angstrom)
hist = sc.histogram(dspacing_events, dspacing_bins)
sc.show(hist['spectrum', 0:3]['d-spacing', 0:7])

In [ ]:
plot(hist)

### Summing (focussing) and normalizing

After conversion to `Dim.DSpacing`, generic `sum` and `/` operations can be used to "focus" and normalize the diffraction data to the vanadium run:

In [ ]:
summed = sc.sum(hist, 'spectrum')
plot(summed)

In [ ]:
normalized = summed['sample'] / summed['vanadium']
plot(normalized)

### Focussing with $\theta$ dependence in event-mode

Instead of focussing all data into a single spectrum, we can use `groupby` to focus each of multiple groups of spectra into a distinct output spectrum.
Here we define groups based on a range of scattering angles &ndash; a simple $\theta$-dependent binning.
This also demonstrates how we can postpone histogramming until after the focussing step.

In [ ]:
theta = sc.Variable(['scattering_angle'],
                    unit=sc.units.rad,
                    values=np.linspace(0.0, np.pi/2, num=16))
focussed = sc.groupby(
    dspacing_events,
    'scattering_angle',
    bins=theta).flatten('spectrum')
focussed = sc.histogram(focussed, dspacing_bins)
normalized = focussed['sample'] / focussed['vanadium']

In [ ]:
plot(normalized)

As a bonus, we can use slicing and a dict-comprehension to quickly create of plot comparing the spectra for different scattering angle bins:

In [ ]:
# compute centers of theta bins
angles = normalized.coords['scattering_angle'].values
angles = 0.5*(angles[1:] + angles[:-1])
plot(sc.Dataset(
    {
        '{}'.format(angles[group]):
        normalized['d-spacing', 300:500]['scattering_angle', group]
        for group in range(2,6)
    }))